# AR6-based forcing

Based on `notebooks/make-forcing.ipynb`
at https://github.com/ClimateIndicator/forcing-timeseries

In [1]:
import warnings
import numpy as np
import pandas as pd
from scipy import stats, optimize
import h5py
from tables import NaturalNameWarning
from mce.util.io import RetrieveGitHub
from mce.core.forcing_ar6 import InputSeries, RfAllAR6

In [2]:
warnings.simplefilter('ignore', NaturalNameWarning)

In [3]:
repo = RetrieveGitHub('ClimateIndicator', 'forcing-timeseries', 'datain')

In [4]:
outpath = 'untracked/datain_2024.h5'
dname = 'ClimateIndicator-2024/forcing-timeseries'

## Uncertainty ranges

In [5]:
SAMPLES = 100000
NINETY_TO_ONESIGMA = stats.norm.ppf(0.95)
# Required adjustment to each species to ensure overall halogenated gas ERF uncertainty is around 19%
HALOGEN_SCALING = 2.05

In [6]:
species_halogen = [
    # hydrofluorocarbons (HFC) and fully fluorinated species including perfluorocarbons (PFCs)
    'HFC-125', 'HFC-134a', 'HFC-143a', 'HFC-152a', 'HFC-227ea', 'HFC-23',
    'HFC-236fa', 'HFC-245fa', 'HFC-32', 'HFC-365mfc', 'HFC-43-10mee',
    'NF3', 'C2F6', 'C3F8', 'n-C4F10', 'n-C5F12', 'n-C6F14', 'i-C6F14',
    'C7F16', 'C8F18', 'CF4', 'c-C4F8', 'SF6', 'SO2F2', 'CCl4',
    # chlorofluorocarbons (CFC), hydrofluorochlorocarbons (HCFC),
    # chlorocarbons, hydrochlorocarbons,
    # bromocarbons, hydrobromocarbons, and halons
    'CFC-11', 'CFC-112', 'CFC-112a', 'CFC-113', 'CFC-113a', 'CFC-114',
    'CFC-114a', 'CFC-115', 'CFC-12', 'CFC-13', 'CH2Cl2',
    'CH3Br', 'CH3CCl3', 'CH3Cl', 'CHCl3',
    'HCFC-124', 'HCFC-133a', 'HCFC-141b', 'HCFC-142b', 'HCFC-22', 'HCFC-31',
    'Halon-1211', 'Halon-1301', 'Halon-2402',
]
len(species_halogen)

49

In [7]:
# uncertainties from IPCC
uncertainty_seed = 38572

unc_ranges = pd.Series({
    k: v / NINETY_TO_ONESIGMA
    for k, v in {
        'CO2': 0.12,
        'CH4': 0.20,
        'N2O': 0.14,
        'O3': 0.50, # total ozone
        'H2O_stratospheric': 1.00, # stratospheric WV from CH4
        'contrails': 0.70, # contrails approx - half-normal
        'BC_on_snow': 1.25, # bc on snow - half-normal
        'land_use': 0.50, # land use change
        'volcanic': 0.25,
        'solar': 0.5, # solar (amplitude)
        **{
            k1: 0.26 * HALOGEN_SCALING if k1 in [
                'HFC-152a', 'CH2Cl2', 'CH3Cl', 'CHCl3',
                'HCFC-133a', 'HCFC-31',
            ] else 0.19 * HALOGEN_SCALING
            for k1 in species_halogen
        },
    }.items()
})

In [8]:
unc_ranges = unc_ranges.reindex(
    ['CO2', 'CH4', 'N2O']
    + species_halogen
    + [
        'O3', 'H2O_stratospheric', 'contrails', 'BC_on_snow', 'land_use',
        'volcanic', 'solar',
    ]
)

In [9]:
n = len(unc_ranges)
scale_df = pd.DataFrame(
    stats.norm.rvs(
        size=(SAMPLES, n), 
        loc=np.ones((SAMPLES, n)), 
        scale=unc_ranges.values, 
        random_state=uncertainty_seed,
    ),
    columns=unc_ranges.index,
)

In [10]:
def f_opt(x, q05_desired, q50_desired, q95_desired):
    "x is (a, loc, scale) in that order."
    q05, q50, q95 = stats.skewnorm.ppf(
        (0.05, 0.50, 0.95), x[0], loc=x[1], scale=x[2]
    )
    return (q05 - q05_desired, q50 - q50_desired, q95 - q95_desired)

lapsi_params = optimize.root(f_opt, [1, 1, 1], args=(0, 1, 2.25)).x
contrails_params = optimize.root(f_opt, [1, 1, 1], args=(19/57, 1, 98/57)).x

In [11]:
# skewnormal for asymmetric distributions
scale_df['BC_on_snow'] = stats.skewnorm.rvs(
    lapsi_params[0],
    loc=lapsi_params[1],
    scale=lapsi_params[2],
    size=SAMPLES,
    random_state=3701584,
)

scale_df['contrails'] = stats.skewnorm.rvs(
    contrails_params[0],
    loc=contrails_params[1],
    scale=contrails_params[2],
    size=SAMPLES,
    random_state=3701585,
)

In [12]:
trend_solar = pd.Series(stats.norm.rvs(
    size=SAMPLES, 
    loc=0., 
    scale=0.07/NINETY_TO_ONESIGMA, 
    random_state=uncertainty_seed,
))

In [13]:
scale_df.to_hdf(outpath, key=f'{dname}/scale_df')
trend_solar.to_hdf(outpath, key=f'{dname}/trend_solar')

In [14]:
with h5py.File(outpath, 'r+') as f1:
    dset = f1[f'{dname}/scale_df']
    dset.attrs['description'] = 'Forcing uncertainty ensemble of scale factors by agent'

    dset = f1[f'{dname}/trend_solar']
    dset.attrs['description'] = 'Forcing uncertainty ensemble of solar trend'

## Input data

In [13]:
datain = {}

In [14]:
path = repo.retrieve('data/gcp_emissions/gcp_2023.csv')
df = pd.read_csv(path, index_col=0)
df.columns = pd.MultiIndex.from_tuples([
    (f'CO2 {x}', 'Gt C/yr') for x in df.columns
])
datain['emis_co2'] = df.rename_axis(None)

[2024-12-12 15:52:07 mce.util.io] INFO:Use local file datain/ClimateIndicator/forcing-timeseries/data/gcp_emissions/gcp_2023.csv retrieved from https://github.com/ClimateIndicator/forcing-timeseries/raw/main/data/gcp_emissions/gcp_2023.csv on 2024-10-30


In [15]:
# notebooks/trace-gas-global-mean.ipynb
path = repo.retrieve('output/ghg_concentrations_1750-2023.csv')
df = pd.read_csv(path, index_col=0)

# Linear interpolation between 1750 and 1850
df = df.reindex(np.arange(df.index[0], df.index[-1]+1)).interpolate()
df.columns = pd.MultiIndex.from_tuples([
    (gas, {'CO2': 'ppm', 'CH4': 'ppb', 'N2O': 'ppb'}.get(gas, 'ppt'))
    for gas in df.columns
])
datain['conc_ghg'] = df.rename_axis(None)

[2024-12-12 15:52:08 mce.util.io] INFO:Use local file datain/ClimateIndicator/forcing-timeseries/output/ghg_concentrations_1750-2023.csv retrieved from https://github.com/ClimateIndicator/forcing-timeseries/raw/main/output/ghg_concentrations_1750-2023.csv on 2024-10-30


In [16]:
# slcf-emissions.ipynb
path = repo.retrieve('output/slcf_emissions_1750-2023.csv')
df = pd.read_csv(path, index_col=0)

# NOx is scaled to NO2
df.columns = pd.MultiIndex.from_tuples([
    (x, f'Mt {x}/yr') for x in df.columns
])
datain['emis_slcf'] = df

[2024-12-12 15:52:10 mce.util.io] INFO:Use local file datain/ClimateIndicator/forcing-timeseries/output/slcf_emissions_1750-2023.csv retrieved from https://github.com/ClimateIndicator/forcing-timeseries/raw/main/output/slcf_emissions_1750-2023.csv on 2024-10-31


In [17]:
path = repo.retrieve('data/ar6/solar_erf.csv')
datain['erf_solar'] = (
    pd.read_csv(path, index_col=0)
    .squeeze()
    .rename_axis(None)
    .rename(None)
    .loc[1750:2023]
)

[2024-12-12 15:52:11 mce.util.io] INFO:Use local file datain/ClimateIndicator/forcing-timeseries/data/ar6/solar_erf.csv retrieved from https://github.com/ClimateIndicator/forcing-timeseries/raw/main/data/ar6/solar_erf.csv on 2024-10-31


In [18]:
path = repo.retrieve('output/volcanic_sAOD_ERF_annual_-9500-2023.csv')
datain['erf_volcanic'] = (
    pd.read_csv(path, index_col=0)
    .loc[1750.0:2023.5, 'volcanic_ERF']
    .rename(int)
    .rename(None)
    .rename_axis(None)
)

[2024-12-12 15:52:12 mce.util.io] INFO:Use local file datain/ClimateIndicator/forcing-timeseries/output/volcanic_sAOD_ERF_annual_-9500-2023.csv retrieved from https://github.com/ClimateIndicator/forcing-timeseries/raw/main/output/volcanic_sAOD_ERF_annual_-9500-2023.csv on 2024-10-31


In [19]:
# contrails
# depends on IEA Word Oil Statistics 2023
path = repo.retrieve('output/contrails_ERF_1930-2023.csv')
d1 = pd.read_csv(path, index_col=0).squeeze().rename_axis(None).rename(None)
datain['erf_contrails'] = d1.reindex(np.arange(1750, 2024), fill_value=0.)

[2024-12-12 15:52:16 mce.util.io] INFO:Use local file datain/ClimateIndicator/forcing-timeseries/output/contrails_ERF_1930-2023.csv retrieved from https://github.com/ClimateIndicator/forcing-timeseries/raw/main/output/contrails_ERF_1930-2023.csv on 2024-11-21


In [20]:
# aerosols calibration results
path = repo.retrieve('data/ar6/table_mean_thornhill_collins_orignames.csv')
df = pd.read_csv(path, index_col=0)
# map_name = {'HC': 'EESC', 'VOC': 'NMVOC'}
map_name = {'HC': 'ODS', 'VOC': 'NMVOC'}
datain['ari_emitted'] = (
    df['Aerosol']
    .rename(map_name)
    .rename(None)
    .rename_axis(None)
)

[2024-12-12 15:52:18 mce.util.io] INFO:Use local file datain/ClimateIndicator/forcing-timeseries/data/ar6/table_mean_thornhill_collins_orignames.csv retrieved from https://github.com/ClimateIndicator/forcing-timeseries/raw/main/data/ar6/table_mean_thornhill_collins_orignames.csv on 2024-10-31


In [21]:
path = repo.retrieve('data/ar6/table_std_thornhill_collins_orignames.csv')
df = pd.read_csv(path, index_col=0)
datain['ari_emitted_std'] = (
    df['Aerosol_sd']
    .rename(map_name)
    .rename(None)
    .rename_axis(None)
)

[2024-12-12 15:52:21 mce.util.io] INFO:Use local file datain/ClimateIndicator/forcing-timeseries/data/ar6/table_std_thornhill_collins_orignames.csv retrieved from https://github.com/ClimateIndicator/forcing-timeseries/raw/main/data/ar6/table_std_thornhill_collins_orignames.csv on 2024-10-31


In [22]:
path = repo.retrieve('data/fair-calibrate-1.4.1/aerosol_cloud.csv')
datain['aci_cal'] = pd.read_csv(path, index_col=0)

[2024-12-12 15:52:23 mce.util.io] INFO:Use local file datain/ClimateIndicator/forcing-timeseries/data/fair-calibrate-1.4.1/aerosol_cloud.csv retrieved from https://github.com/ClimateIndicator/forcing-timeseries/raw/main/data/fair-calibrate-1.4.1/aerosol_cloud.csv on 2024-10-31


In [23]:
# observed GMST
path = repo.retrieve('data/gmst/IGCC_GMST_1850-2022.csv')
datain['temp_obs'] = (
    pd.read_csv(path, index_col=0)
    .squeeze()
    .rename(int)
    .rename(None)
    .rename_axis(None)
)

[2024-12-12 15:52:24 mce.util.io] INFO:Use local file datain/ClimateIndicator/forcing-timeseries/data/gmst/IGCC_GMST_1850-2022.csv retrieved from https://github.com/ClimateIndicator/forcing-timeseries/raw/main/data/gmst/IGCC_GMST_1850-2022.csv on 2024-11-21


In [24]:
# ozone calibration results
for k in ['trop', 'strat']:
    path = repo.retrieve(f'data/ozone/skeie_ozone_{k}.csv')
    datain[f'skeie_ozone_{k}'] = (
        pd.read_csv(path, index_col=0)
        .rename(int, axis=1)
        .rename_axis(None)
    )

[2024-12-12 15:52:26 mce.util.io] INFO:Use local file datain/ClimateIndicator/forcing-timeseries/data/ozone/skeie_ozone_trop.csv retrieved from https://github.com/ClimateIndicator/forcing-timeseries/raw/main/data/ozone/skeie_ozone_trop.csv on 2024-11-21
[2024-12-12 15:52:26 mce.util.io] INFO:Use local file datain/ClimateIndicator/forcing-timeseries/data/ozone/skeie_ozone_strat.csv retrieved from https://github.com/ClimateIndicator/forcing-timeseries/raw/main/data/ozone/skeie_ozone_strat.csv on 2024-11-21


In [27]:
for k, v in datain.items():
    v.to_hdf(outpath, key=f'{dname}/{k}')

In [28]:
with h5py.File(outpath, 'r+') as f1:
    for k, v in {
        'emis_co2': 'Emissions of CO2 in 1750-2023',
        'conc_ghg': 'Concentrations of GHGs in 1750-2023',
        'emis_slcf': 'Emissions of SLCFs in 1750-2023',
        'erf_solar': 'Effective radiative forcing of solar irradiance in 1750-2023',
        'erf_volcanic': 'Effective radiative forcing of volcanic activity in 1750-2023',
        'erf_contrails': 'Effective radiative forcing of contrails and contrail-induced cirrus in 1750-2023',
        'ari_emitted': 'Reference radiative forcing by SLCF species',
        'ari_emitted_std': 'Standard deviations of reference radiative forcing by SLCF species',
        'aci_cal': 'Forcing uncertainty ensemble related to aerosol-cloud interactions',
        'temp_obs': 'Temperature time series used for temperature feedback of ozone forcing',
        'skeie_ozone_trop': 'Reference radiative forcing of tropospheric ozone',
        'skeie_ozone_strat': 'Reference radiative forcing of stratospheric ozone',
        }.items():
        grp = f1[f'{dname}/{k}']
        grp.attrs['description'] = v

## Forcing assessment

In [25]:
def df_concat(datain, *args, **kw):
    map_name = kw.get('map_name', {})
    ret_slice = kw.get('ret_slice', False)

    data = [
        (
            datain[k].to_frame(map_name.get(k, k))
            if isinstance(datain[k], pd.Series)
            else datain[k]
        )
        for k in args
    ]
    df = pd.concat(data, axis=1)

    if ret_slice:
        jn = np.hstack([0, [df.shape[1] for df in data]]).cumsum()
        map_slc = dict(zip(
            args, [slice(jn[i-1], jn[i]) for i in range(1, len(jn))],
        ))
        return df, map_slc
    else:
        return df

In [26]:
df_emis = df_concat(datain, 'emis_co2', 'emis_slcf')

In [27]:
df_emis_co2__cumsum = (
    datain['emis_co2']
    .cumsum(axis=0)
    .rename(lambda x: x.replace('/yr', ''), axis=1, level=1)
)

In [28]:
map_erf_name = {
    f'erf_{x}': x for x in ['contrails', 'solar', 'volcanic']
}
df_erf = df_concat(datain, *list(map_erf_name), map_name=map_erf_name)

In [29]:
years = np.arange(1750, 2020)
dfin_ref = pd.concat({
    k:
    df.loc[years].T
    .rename_axis(['name', 'unit'])
    .reset_index()
    .set_index('name')
    for k, df in zip(
        ['emis', 'emis_co2__cumsum', 'conc'],
        [df_emis, df_emis_co2__cumsum, datain['conc_ghg']],
    )
}).rename_axis((None, None))

In [30]:
year_pi = 1750
din = InputSeries(erf=df_erf)
obj = RfAllAR6(dfin_ref, din)

In [31]:
obj.init__ari(datain['ari_emitted'], datain['ari_emitted_std'])
obj.init__aci(datain['aci_cal'])
obj.init__o3(
    datain['temp_obs'],
    datain['skeie_ozone_trop'],
    datain['skeie_ozone_strat'],
)

In [32]:
df_conc_ghg = datain['conc_ghg'].droplevel(1, axis=1)
df_emis_slcf = datain['emis_slcf'].droplevel(1, axis=1)
df_emis_co2 = datain['emis_co2'].droplevel(1, axis=1)

In [33]:
idx_time = df_conc_ghg.index

In [34]:
forcing = {}
forcing_ens = {}

In [35]:
# contrails, solar, and volcanics
for k, v in obj.erf__others().items():
    forcing[k] = v.rename(None).values

In [36]:
cat = 'solar'
# as AR6, trend extended to 2023, but with 2019 end
d1 = pd.Series(np.nan, index=idx_time)
d1.loc[1750] = 0.
d1.loc[2019:] = 1.
d1 = d1.interpolate()
forcing_ens[cat] = (
    d1.values[:, None] * trend_solar.values[None, :]
    + forcing[cat][:, None] * scale_df[cat].values[None, :]
)

In [37]:
cat = 'aerosol-radiation_interactions'
forcing[cat] = obj.erf__ari(df_emis_slcf, df_conc_ghg).values

dfin = obj.save__ari_dfin
radeff_ens = obj.ari__radeff_ens[dfin.keys()]
forcing_ens[cat] = (
    dfin.values[:, None, :] * radeff_ens.values[None, :, :]
).sum(axis=2)

In [38]:
cat = 'aerosol-cloud_interactions'
forcing[cat] = obj.erf__aci(df_emis_slcf).values
forcing_ens[cat] = obj.save__aci_ens.values

In [39]:
cat = 'land_use'
forcing[cat] = obj.erf__land_use(df_emis_co2).values

In [40]:
cat = 'BC_on_snow'
forcing[cat] = obj.erf__bc_on_snow(df_emis_slcf).values

In [41]:
# Greenhouse gas concentrations
df_erf_ghgs = obj.erf__ghg(df_conc_ghg, agg_minor=False)
df_erf_ghgs_agg = obj.erf__ghg(df_conc_ghg, agg_minor='halogen')

for k, v in df_erf_ghgs.iteritems():
    forcing[k] = v.rename(None).values

In [42]:
cat = 'O3'
forcing[cat] = obj.erf__o3(df_emis_slcf, df_conc_ghg).values

In [43]:
cat = 'H2O_stratospheric'
forcing[cat] = obj.erf__h2o_strat(df_erf_ghgs).values

## Comparison with original output

In [44]:
path = repo.retrieve('output/ERF_best_1750-2023.csv')
df_erf_ref = pd.read_csv(path, index_col=0)

df = {}
for x in ['best', 'p05', 'p95']:
    path = repo.retrieve(f'output/ERF_{x}_aggregates_1750-2023.csv')
    df[x] = pd.read_csv(path, index_col=0)

df_erf_agg_ref = pd.concat(df, axis=1)

[2024-12-12 15:54:39 mce.util.io] INFO:Use local file datain/ClimateIndicator/forcing-timeseries/output/ERF_best_1750-2023.csv retrieved from https://github.com/ClimateIndicator/forcing-timeseries/raw/main/output/ERF_best_1750-2023.csv on 2024-10-31
[2024-12-12 15:54:39 mce.util.io] INFO:Use local file datain/ClimateIndicator/forcing-timeseries/output/ERF_best_aggregates_1750-2023.csv retrieved from https://github.com/ClimateIndicator/forcing-timeseries/raw/main/output/ERF_best_aggregates_1750-2023.csv on 2024-10-11
[2024-12-12 15:54:39 mce.util.io] INFO:Use local file datain/ClimateIndicator/forcing-timeseries/output/ERF_p05_aggregates_1750-2023.csv retrieved from https://github.com/ClimateIndicator/forcing-timeseries/raw/main/output/ERF_p05_aggregates_1750-2023.csv on 2024-12-02
[2024-12-12 15:54:39 mce.util.io] INFO:Use local file datain/ClimateIndicator/forcing-timeseries/output/ERF_p95_aggregates_1750-2023.csv retrieved from https://github.com/ClimateIndicator/forcing-timeseries/r

In [45]:
df = pd.DataFrame(forcing, index=idx_time)
np.allclose(df, df_erf_ref[df.columns])

True

In [46]:
cats = [
    'CO2',
    'CH4',
    'N2O',
    'halogen',
    'O3',
    'aerosol-radiation_interactions',
    'aerosol-cloud_interactions',
    'land_use',
    'BC_on_snow',
    'H2O_stratospheric',
    'contrails',
    'solar',
    'volcanic',
]
pd.concat([
    df.drop(species_halogen, axis=1),
    df_erf_ghgs_agg[['halogen']],
], axis=1)[cats]

,CO2,CH4,N2O,halogen,O3,aerosol-radiation_interactions,aerosol-cloud_interactions,land_use,BC_on_snow,H2O_stratospheric,contrails,solar,volcanic
1750,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000,0.000000,0.097590,0.246412
1751,0.001417,0.000508,0.000074,7.958010e-10,-0.000383,0.000312,0.003303,-0.000007,-0.000366,0.000047,0.000000,0.079244,0.246420
1752,0.002833,0.001016,0.000148,1.591602e-09,-0.000318,0.000621,0.004770,-0.000020,-0.000493,0.000093,0.000000,0.049166,0.246422
1753,0.004249,0.001524,0.000222,2.387403e-09,-0.000089,0.000006,0.002776,-0.000037,-0.000379,0.000140,0.000000,0.012988,0.246423
1754,0.005665,0.002031,0.000296,3.183204e-09,0.000722,-0.001207,-0.002953,-0.000059,0.000028,0.000187,0.000000,-0.021427,-0.116729
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019,2.157749,0.543972,0.208488,4.081485e-01,0.483879,-0.242714,-0.903946,-0.200000,0.080000,0.050000,0.061541,-0.022173,0.133221
2020,2.190500,0.548546,0.212231,4.087011e-01,0.466118,-0.194643,-0.791027,-0.200956,0.073532,0.050420,0.030676,-0.016969,0.096304
2021,2.221573,0.554765,0.216261,4.096640e-01,0.483894,-0.228537,-0.873016,-0.201917,0.079076,0.050992,0.036909,0.019557,0.153448
2022,2.253999,0.560691,0.220281,4.106025e-01,0.475106,-0.198474,-0.766242,-0.202876,0.069397,0.051537,0.040946,0.059393,0.095918


In [47]:
for cat, ens in forcing_ens.items():
    print(
        cat,
        np.allclose(
            np.percentile(ens, [5, 95], axis=1).T,
            df_erf_agg_ref[[('p05', cat), ('p95', cat)]]
        ),
    )

solar True
aerosol-radiation_interactions True
aerosol-cloud_interactions True


In [48]:
for cat, d1 in forcing.items():
    if cat in list(forcing_ens) + species_halogen:
        continue

    ens = d1[:, None] * scale_df[cat].values[None, :]
    print(
        cat,
        np.allclose(
            np.percentile(ens, [5, 95], axis=1).T,
            df_erf_agg_ref[[('p05', cat), ('p95', cat)]]
        ),
    )

contrails True
volcanic True
land_use True
BC_on_snow True
CO2 True
CH4 True
N2O True
O3 True
H2O_stratospheric True


In [49]:
ens = 0.

for cat, d1 in forcing.items():
    if cat not in species_halogen:
        continue

    ens += d1[:, None] * scale_df[cat].values[None, :]

cat = 'halogen'
np.allclose(
    np.percentile(ens, [5, 95], axis=1).T,
    df_erf_agg_ref[[('p05', cat), ('p95', cat)]]
)

True

## Saved data

In [55]:
with h5py.File(outpath, 'r') as f1:
    grp = f1[dname]
    grp.visititems(
        lambda name, obj:
        print('{}: {}'.format(name, obj.attrs['description']))
        if isinstance(obj, h5py.Group) else None
    )

aci_cal: Forcing uncertainty ensemble related to aerosol-cloud interactions
ari_emitted: Reference radiative forcing by SLCF species
ari_emitted_std: Standard deviations of reference radiative forcing by SLCF species
conc_ghg: Concentrations of GHGs in 1750-2023
emis_co2: Emissions of CO2 in 1750-2023
emis_slcf: Emissions of SLCFs in 1750-2023
erf_contrails: Effective radiative forcing of contrails and contrail-induced cirrus in 1750-2023
erf_solar: Effective radiative forcing of solar irradiance in 1750-2023
erf_volcanic: Effective radiative forcing of volcanic activity in 1750-2023
scale_df: Forcing uncertainty ensemble of scale factors by agent
skeie_ozone_strat: Reference radiative forcing of stratospheric ozone
skeie_ozone_trop: Reference radiative forcing of tropospheric ozone
temp_obs: Temperature time series used for temperature feedback of ozone forcing
trend_solar: Forcing uncertainty ensemble of solar trend
